In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

#### Reading in Audio Features

In [2]:
Compare = pd.read_pickle('../Pickle_Files/tracks.pkl')
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
track_elems = pd.read_pickle('../Pickle_Files/track_elems.pkl')

#### Drops all uncommon songs

In [3]:
Compare = pd.merge(audio_features[['track_uri']],Compare,on='track_uri')

#### Ordering tracks

In [4]:
Compare = Compare.sort_values('track_uri',ascending=True)

In [5]:
Compare = Compare.reset_index().drop(['index'],axis=1)

In [6]:
len(Compare)

2254670

#### Creating Dataframe 'Cosine'

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
cosine = pd.read_pickle('../Pickle_Files/cosine_dataframe.pkl')
cosine = cosine.drop(['Popularity_Rating_Normalized','principal component 1'],axis=1)

In [9]:
X = cosine.values

#### Normalizing values before cosine calculated

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit(cosine.values).transform(cosine.values)

# Recommender System
<br>-  This Recommender System takes in an input dataframe and makes a recommendation using the cosine similarity of the provided features.
<br>-  It chooses N recommendations of the length of the training set and uses ".sample()" to pick a random number of recommendations the same length as the test set.
<br>-  It then calculates the precision and recall of the test set.

The definition below calculates the precision and recall of a given recommender system

In [11]:
def recommender_precision(predicted, actual):
    def calc_precision(predicted, actual):
        prec = [value for value in predicted if value in actual]
        prec = np.round(float(len(prec)) / float(len(predicted)), 4)
        return prec

    precision = np.mean(list(map(calc_precision, predicted, actual)))
    return precision


def recommender_recall(predicted, actual):
    def calc_recall(predicted, actual):
        reca = [value for value in predicted if value in actual]
        reca = np.round(float(len(reca)) / float(len(actual)), 4)
        return reca

    recall = np.mean(list(map(calc_recall, predicted, actual)))
    return recall

#### Creating Two Empty Lists, Precision and Recall
<br> These are used in the Recommender System for calculations

In [12]:
Precision = []
Recall = []

In [13]:
def Recommender_System(df):
    
    # Obtaining index values to later compare 
    
    df = pd.merge(df,Compare['track_uri'].reset_index(),on='track_uri')
    
    #df = df.set_index('index')

    #df = df.reset_index()

    df = df[['track_uri','index']]

    df = pd.merge(cosine,df,on='track_uri')

    df = df.set_index('index')

    df = df.drop('track_uri',axis=1)

    df_values = df.index.values

    # Creates training and test set of .7 and .3

    msk = np.random.rand(len(df)) < 0.7

    dftrain = df[msk]

    dftest = df[~msk]

    dfvalues = dftrain.index.values # Index values used in cosine similarity
    
    num_tracks_desired = 1 ## Recommends only one song
    def rec_top_n(n):
        return np.argsort(cosine_similarity(X, X[dfvalues]), axis=0)[::-1][1:n] # Cosine similarity finds closest to each song
    # Prints index of all recommended songs
    print(rec_top_n(num_tracks_desired+1))
    test = rec_top_n(num_tracks_desired+1).flatten()

    test = Compare.iloc[test]

    testset = Compare[['track_name','track_uri','artist_name']].iloc[dftest.index]
    
    prediction = test.sample(len(dftest))
    prediction = prediction[['track_name','track_uri','artist_name']]
    prediction.columns = ['track_name_pred', 'track_uri_pred','artist_name_pred']

    # Evalution

    pred = [prediction.index.values]
    # Uses index to check predicted against actual
    actual = [dftest.index.values] 

    #returns Precision 
    print('\nPrecision: ' +str(recommender_precision(pred,actual)))
    Precision.append(recommender_precision(pred,actual))
    #returns Recall
    print('\nRecall: ' +str(recommender_recall(pred,actual)))
    Recall.append(recommender_recall(pred,actual))
    
    # Returns songs predicted
    
    prediction = prediction.reset_index()
    testset = testset.reset_index()
    prediction = prediction.drop(['index'],axis=1)
    testset = testset.drop(['index'],axis=1)
    final = testset.join(prediction)
    return final

### Example Recommending for Playlist 0 (pid0)

In [14]:
pid0 = track_elems[track_elems['pid']==0]

In [15]:
pid0 = pd.merge(Compare,pid0[['pid','track_uri']],on='track_uri')

In [16]:
df = pid0

In [17]:
Recommender_System(pid0)

[[1100597  607435  204895 1179543 1767356 1109322  176447 1394716 1893584
   255086 2252144 1155088 1662541 1157312 1065742 2177488 1451916 2127867
   327791  258218  430537 1132502 2246260 1622293  226786  539920 1004056
  1407934 2222744  764600  451615  475954  106982  825585   45972 1471741
    31687]]

Precision: 0.0

Recall: 0.0


,track_name,track_uri,artist_name,track_name_pred,track_uri_pred,artist_name_pred
0,Yeah!,spotify:track:0XUfyU2QviPAs6bxSpXYG4,Usher,Imagina,spotify:track:3pI91kPGj5SgERZu4bhnGE,Chambao
1,Year 3000,spotify:track:1TfAhjzRBWzYZ8IdUV3igl,Jonas Brothers,Cool for the Summer - 135 BPM,spotify:track:1YO8Bwlq2FxfAJSFOZV6v5,Hanna
2,Livin on Sunday,spotify:track:1X5WGCrUMuwRFuYU1eAo2I,Cris Cab,Caroline,spotify:track:7Ltygt0h1c9oam7jZiKgCB,About-face
3,Paper Planes,spotify:track:1kusepF3AacIEtUTYrw4GV,M.I.A.,Gente de la Gente,spotify:track:0beph65hikTCRqrh93Mvbb,Los Buchones de Culiacan
4,Leavin',spotify:track:20ORwCJusz4KS2PbTPVNKo,Jesse McCartney,Ven Muñeca - En Vivo Desde Valle De Bravo/2012,spotify:track:7tCQP3jweE3yT1BlhJns7t,Tierra Cali
5,Buttons,spotify:track:3BxWKCI06eQ5Od8TY2JBeA,The Pussycat Dolls,Fill Us Up With Your Mercy,spotify:track:3z4WJEc8dH8nY4FEDl3vex,Garnett Silk
6,Forever,spotify:track:3H1LCvO3fVsK2HPguhbml0,Chris Brown,Duracell,spotify:track:2qMLuty4qaxaSsvYNbMmGi,CEONRPG
7,Right Where You Want Me - Radio Edit Version,spotify:track:4Y45aqo9QMa57rDsAJv40A,Jesse McCartney,Antidoto,spotify:track:1rGfmrqwm1RSa5vlHIRfuL,Jaime Viñas
8,"Jumpin', Jumpin'",spotify:track:4pmc2AxSEq6g7hPVlJCPyP,Destiny's Child,The Way,spotify:track:2dJIq2l8nlfpIVd8WP4alk,Egypt Central
9,Party In The U.S.A.,spotify:track:5Q0Nhxo0l2bP3pNjpGJwV1,Miley Cyrus,One More Time,spotify:track:54vt1J2KJWX8Atntpgn1kB,Enova


### Example with Popular DJs
<a id='pop_djs'></a>

In [18]:
one = Compare[Compare['artist_name']=='Avicii'].sample(5) 
two = Compare[Compare['artist_name']=='Hardwell'].sample(5)   
three = Compare[Compare['artist_uri']==('spotify:artist:2CIMQHirSU0MQqyYHq0eOx')].sample(5)#deadmau5
four = Compare[Compare['artist_name']=='Swedish House Mafia'].sample(5)   
five = Compare[Compare['artist_name']=='Calvin Harris'].sample(5)  

In [19]:
cols = five.columns

all_songs = pd.concat([one.head(5), two.head(5),three.head(5),four.head(5),five.head(5)], ignore_index=False)

In [20]:
all_songs.sample(5)

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,seconds,minutes
563186,spotify:track:1wGRwJdWJ92jdEbGF1TXZw,A State Of Trance Episode 731,spotify:album:5Mdgx8Sl0o1Z5S4yvxsk4s,Hardwell,spotify:artist:6BrvowZBreEkXzJQMpL174,248164,None,Off The Hook (ASOT 731) [Trending Track] - Ori...,248.164,4.136067
340580,spotify:track:1AdfHX1DE79Q6walr15N7c,United We Are,spotify:album:3MnBKsNCW5X5Qyi5GSVgPk,Hardwell,spotify:artist:6BrvowZBreEkXzJQMpL174,339375,None,Birds Fly - W&W Festival Mix Edit,339.375,5.656250
490673,spotify:track:1gkeTTfkJK7ZsJS3TWVVzu,Ultra 2012,spotify:album:3HuYl98gT0ixfoBHzFsVgw,deadmau5,spotify:artist:2CIMQHirSU0MQqyYHq0eOx,484986,None,Raise Your Weapon,484.986,8.083100
1482454,spotify:track:57E15uZh5L8EW9fCwpDTbR,Antidote,spotify:album:774xaOL4PAAtvHkyPWRikH,Swedish House Mafia,spotify:artist:1h6Cn3P4NGzXbaXidqURXs,353492,None,Antidote - Tommy Trash Remix,353.492,5.891533
1512793,spotify:track:5DiZlb1Xb15zOCjgvJssse,Waiting For Love,spotify:album:6zGYMf7735htmC3XNQmONX,Avicii,spotify:artist:1vCWHaC5f2uS3yhpwWbIA6,224505,None,Waiting For Love - Fabich Remix,224.505,3.741750


<br>-Here we can see that the recommender system has been able to identify that the songs in the playlist were DJ songs using only the audio features.
<br>-It has recommended three remixes and four songs from other DJs, this shows by just using the audio features it can make good recommendations

In [21]:
Recommender_System(all_songs)

[[ 218742 1519211  427732 1217655 1483173 1626351 1299815  800360 1078786
  1702221 1084482  631008 1509656  372931  840267 2198181  111358  859050]]

Precision: 0.0

Recall: 0.0


,track_name,track_uri,artist_name,track_name_pred,track_uri_pred,artist_name_pred
0,Raise Your Weapon - Maywald Remix,spotify:track:0GDw3YsRk9avS29BupciRX,deadmau5,Addiction,spotify:track:2xVhDUX6s4YJKIwYE7JHeR,Sin Cos Tan
1,UMF (Ultra Music Festival Anthem),spotify:track:1OIap2AeiVhAavWyRKTSba,Avicii,"Hey, Soul Sister - Humanjive Remix",spotify:track:2Am9XjlQnvCxqCraxCGiQO,Db Sound
2,Feel So Close,spotify:track:1oRpmxf7EeAOoYb3ORzu99,Calvin Harris,DalePlay (5) La Quemada De La Noche,spotify:track:0kmjolyAqGMwBIyboSHRJJ,Dj Towa
3,I Could Be The One [Avicii vs Nicky Romero] - ...,spotify:track:2bNrvWD9d5Zn9tS8XvcaMk,Avicii,Rising Up,spotify:track:3jx2VeOcZrNe2q4bIkxyWI,Taiki Nulight
4,Run Wild - Manse Radio Mix,spotify:track:543Wd7jKRUs8G39ividPfH,Hardwell,Meditation - Palm Beach Mix,spotify:track:2kveDiWIFDbPXFI4ANvRXp,Groove Chiller
5,Save the World - Zedd Remix,spotify:track:5XZrF0we6XyC3MXJMxXhEU,Swedish House Mafia,7 feet under the keel,spotify:track:1HXyZrujuLB6SGBgwUptN1,7 Feet under the keel
6,Blue,spotify:track:6cZ6H2Jq4LSOqJs9pTZpi0,Calvin Harris,Don't Leave Me - Club Edit,spotify:track:5cAQf0ZSez1MxxVNKcFC8K,Activa


## Precision and Recall on 50 Random Playlists

#### Removing Playlists with less than 20 songs

In [22]:
track_elems = track_elems[track_elems['COUNT(*)']>=1500]

In [23]:
remove = track_elems.groupby('pid').nunique()

In [24]:
new_track_elems = remove[remove['COUNT(*)']>20]

In [25]:
new_track_elems = new_track_elems.drop(['pid','pos','track_uri'],axis=1)

In [26]:
new_track_elems.columns = ['num_tracks']

In [27]:
new_track_elems = pd.merge(track_elems,new_track_elems.reset_index(),on='pid')

#### Picks 50 random playlists

In [28]:
mylist = new_track_elems.pid.unique()

import random

sample_size = 50
sorted_sample = [mylist[i] for i in sorted(random.sample(range(len(mylist)), sample_size))]

df_dict = {name: track_elems.loc[track_elems['pid'] == name] for name in sorted_sample}

#### Sample of one of the playlists

In [29]:
df_dict[sorted_sample[0]].head()

,pid,pos,track_uri,COUNT(*)
7074,751366,79,spotify:track:1XazlqB4YAtvouCmJPWXlM,5757
3006883,751366,16,spotify:track:6ZYS6QQxTLsQ6IFXdVx1r4,19806
4355533,751366,71,spotify:track:047fCsbO4NdmwCBn8pcUXl,11666
4811304,751366,25,spotify:track:7yq4Qj7cqayVTp3FF9CWbm,28448
5453294,751366,17,spotify:track:13HVjjWUZFaWilh2QUJKsP,19688


#### Loop that runs recommendation system for all 100 playlists

In [30]:
x = 0
while x < 50:
    Recommender_System(df_dict[sorted_sample[x]])
    x += 1

[[  69918  580862 1230200 2064687  733749 2094876  540596  401662  713721
  1109870  951561 2093535 1117404 1575989 1997138 1787045 1200473 1441169
  1150629 1730892    1424 1841803   13297  781089 1996977 2236124 1953636]]

Precision: 0.0

Recall: 0.0
[[1950138 1001871  361469  685219  327345 1747499 1866171 1035402 1800469
   156057  239546  776717  312081  489595  473615 2049738 1173653 1876971
  2044352 1291089 1369694 1824232 1069161  643854 2053980  688124 1164268
  2068296 1326549  501526  641518 1400615  813382   64322  357217 1677310
   272547 1621886  474481 1901724 1814410  587304  307974 1473040 1528249
   363706 1625750  857163 1524421 1521855  812574 1334472  965919 2153834
  2249279 1801061  565967 1179603  935555 1112426 1181376 1742708  411252
   121706  805843 1975825 1028387 2119349  702562  587040  854282 1040870
     3286 1611819 1373283  795915 2005250  391045  186041  226650 1054491
   830639 1375979  659892  978163  605732  689804   42261 1061629  272558
  17372


Precision: 0.0

Recall: 0.0
[[2147751   78579 1720642  161207 2087990 2201658 1606961   16048 1370303
   995412  818125 1086683 1086683  955265  810413 1449224 1304282 1048326
   475568  946479  520227]]

Precision: 0.0

Recall: 0.0
[[1058826  607435 1137122 1006620    3919 1136960 2030011  803470 1552686
  2151886  279627 1328572  739919 1440624  658772  399191 1721705 1856449
   376951 1004056  935019  127361  297626  662387 1614172  825585 2121735
  1450237 1817698  848696]]

Precision: 0.0

Recall: 0.0
[[ 275556  651089 1720642 2008242  823069 1180193 1102131 2074823  460171
   220407 1851813 1357697  104129 1030312  609864  982555 1281569  736363
   536396  721707  636843 1549589 1483674 1791296  276756 1922464  313717
  1733685  891630  146205 2070682 1181501 1493457  701769  362869  511581
   977203  521239  249508 1217906  924649  202605 2204176 1439137  916685
  1791108  929110  775261  947451 1926828 1721449  108236  795409 1870802]]

Precision: 0.0

Recall: 0.0
[[ 158748  1


Precision: 0.0

Recall: 0.0
[[2166586  600823  982690  781030  738782  418291 1614942 1777039 1886172
  1997134 2046137  823442 1034709 1199158 2010179 2058762 1691923 1049671
   659569 1125014  741154  211111 1167334 1264938  748075  116268  639165
   697478  329939 1082911  889628 1842646 1020292]]

Precision: 0.0

Recall: 0.0
[[  68512  748533 1140427 2068798 1325637  342183 1985073  684974  243276
   172491  716399 2115503  202640 1327001 1405788 2010662  884479 1459745
  1470780  977886  108236]]

Precision: 0.0

Recall: 0.0


#### Final Precision and Recall

In [31]:
print(Precision)
np_Precision = np.array(Precision)
print('\nMean Precision = '+str(np_Precision.mean())+'\n')

print(Recall)
np_Recall = np.array(Recall)
print('\nMean Recall = '+str(np_Recall.mean()))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Precision = 0.0

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Mean Recall = 0.0


# Conclusion

Although the Precision and Recall for audio features are low it is clear that the recommendations are staying true to the audio features and are recommending audibly similar songs.
<br> As we can see in the <a href='#pop_djs'>Popular DJ</a> example it has picked up that the random playlist were songs by DJs and recommended other remixes
<br> Including Popularity and Clusters should improve these recommendations